| Name | Type (Shape) | Description |
| --- | --- | --- |
| time | array (N,) | time |
| x | array (3,) | $ \mathbf{x} $ |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| Interpolant | list (3,) | Interpolant[0]: Interpolant object for $ u(\mathbf{x}, t)  $ <br /> Interpolant[1]: Interpolant object for $ v(\mathbf{x}, t)  $. <br /> Interpolant[2]: Interpolant object for $ w(\mathbf{x}, t)  $|
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z|
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| trajectory | array (3,N) | integrated trajectory |
| velocity | array (3,N) | velocity along trajectory |
| DF | array (3,3,N) | gradient of flow map at every time-step |

In [2]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add utils folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [3]:
# Import numpy
import numpy as np

# Import package to solve ODE
from scipy.integrate import solve_ivp

# function which evalutes derivative of flow map
from ipynb.fs.defs.dFdt import dFdt

In [2]:
def integration_dFdt(time, x, X, Y, Z, Interpolant, periodic, bool_unsteady):
        
    # time-interval [t0,tN] over which to integrate
    time_interval = [time[0], time[-1]]
        
    # time-array at which trajectory must be evaluated
    t_eval = time.copy()
        
    # Step-size
    dt = time[1]-time[0]
    
    sol = solve_ivp(dFdt, time_interval, x, 'RK45', t_eval, rtol=1e-6, atol=1e-6, first_step = dt, args = (X, Y, Z, Interpolant, periodic, bool_unsteady))
          
    # Initialize velocity array
    velocity = np.zeros((3, len(time)))
            
    for i in range(sol.y.shape[1]-1):
                
        velocity[0, i] = (sol.y[0,i+1]-sol.y[0,i])/dt
        velocity[1, i] = (sol.y[1,i+1]-sol.y[1,i])/dt
        velocity[2, i] = (sol.y[2,i+1]-sol.y[2,i])/dt
        
    trajectory = sol.y.copy()
    
    if periodic[0]:
        
        trajectory[0,:] = trajectory[0,:]%(X[0, -1, 0]-X[0, 0, 0])
    
    if periodic[1]:
        
        trajectory[1,:] = trajectory[1,:]%(Y[-1, 0, 0]-Y[0, 0, 0])
    
    if periodic[2]:
        
        trajectory[2,:] = trajectory[2,:]%(Z[0, 0, -1]-Z[0, 0, 0])
            
    return trajectory, velocity